# Generate training data

In [1]:
import pandas as pd
import json
import re
from typing import List, Tuple, Set

# ============================================================================
# CONFIGURATION
# ============================================================================

# File paths
FILES = {
    'cisco': 'Schemes/Cisco.csv',
    'spanish_ens': 'Schemes/SpanishENS.csv',
    'secnumcloud': 'Schemes/Secnumcloud.csv',
    'bsi_c5': 'Schemes/BSI-C5.json',
    'old_eucs': 'Schemes/OldEucsRequirements.csv',
    'new_eucs': 'Schemes/NewEucsRequirements_with_texts.csv',
    'medina_metrics': 'Schemes/medinaMetrics.csv',
    'fabasoft_metrics': 'Schemes/fabasoftMetrics.csv'
}

# Output files
OUTPUT = {
    'train': 'TrainAndTestData/training_dataset.csv',
    'test': 'TrainAndTestData/test_dataset.csv'
}

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def standardize_id(control_id: str) -> str:
    """
    Standardize control IDs by removing extra whitespace and normalizing format.
    Examples:
    - "OPS-05.3H " -> "OPS-05.3H"
    - "OIS-01 " -> "OIS-01"
    - " BSI C5 OPS-22" -> "OPS-22"
    """
    if pd.isna(control_id) or control_id == '':
        return ''
    
    # Convert to string and strip whitespace
    control_id = str(control_id).strip()
    
    # Remove "BSI C5 " prefix if present
    control_id = re.sub(r'^BSI\s+C5\s+', '', control_id, flags=re.IGNORECASE)
    
    # Remove extra internal whitespace
    control_id = re.sub(r'\s+', ' ', control_id)
    
    return control_id

def parse_control_ids(value: str) -> List[str]:
    """
    Parse a string containing multiple control IDs separated by commas or newlines.
    Returns a list of standardized IDs.
    """
    if pd.isna(value) or value == '':
        return []
    
    # Split by comma or newline
    ids = re.split(r'[,\n]+', str(value))
    
    # Standardize each ID and filter out empty strings
    standardized = [standardize_id(id_str) for id_str in ids]
    return [id_str for id_str in standardized if id_str]

def create_pairs(source_id: str, source_text: str, target_ids: List[str], 
                 target_texts: dict, label: int = 1) -> List[dict]:
    """
    Create training/test pairs from source control to target controls.
    """
    pairs = []
    for target_id in target_ids:
        if target_id in target_texts:
            pairs.append({
                'source_id': source_id,
                'source_text': source_text,
                'target_id': target_id,
                'target_text': target_texts[target_id],
                'label': label
            })
    return pairs

# ============================================================================
# LOAD DATA
# ============================================================================

print("Loading data files...")

# Load Cisco CSV
cisco_df = pd.read_csv(FILES['cisco'], encoding='utf-8')
print(f"✓ Loaded Cisco.csv: {len(cisco_df)} rows")

# Load Spanish ENS
spanish_ens_df = pd.read_csv(FILES['spanish_ens'], encoding='utf-8')
spanish_ens_dict = dict(zip(
    spanish_ens_df['Control ID'].apply(standardize_id),
    spanish_ens_df['Description']
))
print(f"✓ Loaded SpanishENS.csv: {len(spanish_ens_dict)} controls")

# Load SecNumCloud
secnumcloud_df = pd.read_csv(FILES['secnumcloud'], encoding='utf-8')
secnumcloud_dict = dict(zip(
    secnumcloud_df['ID'].apply(standardize_id),
    secnumcloud_df['Description_EN']
))
print(f"✓ Loaded Secnumcloud.csv: {len(secnumcloud_dict)} controls")

# Load BSI-C5 JSON
with open(FILES['bsi_c5'], 'r', encoding='utf-8') as f:
    bsi_c5_data = json.load(f)

bsi_c5_dict = {}
for control in bsi_c5_data:
    code = standardize_id(control.get('code', ''))
    if code and code != 'AAA-00':  # Skip generic control
        desc = control.get('descriptionTitle', '')
        if desc:
            bsi_c5_dict[code] = desc
print(f"✓ Loaded BSI-C5.json: {len(bsi_c5_dict)} controls")

# Load Old EUCS Requirements
old_eucs_df = pd.read_csv(FILES['old_eucs'], encoding='utf-8')
old_eucs_dict = dict(zip(
    old_eucs_df['controlId'].apply(standardize_id),
    old_eucs_df['description']
))
old_eucs_ids = set(old_eucs_dict.keys())
print(f"✓ Loaded OldEucsRequirements.csv: {len(old_eucs_dict)} controls")

# Load New EUCS Requirements
new_eucs_df = pd.read_csv(FILES['new_eucs'], encoding='utf-8')
new_eucs_dict = {}
new_eucs_to_bsi = {}
new_eucs_to_secnum = {}
new_eucs_to_iso27002 = {}
new_eucs_to_iso27017 = {}

for _, row in new_eucs_df.iterrows():
    eucs_id = standardize_id(row['EUCS Ref (Detailed)'])
    eucs_text = row['EUCS Text']
    
    if eucs_id and pd.notna(eucs_text):
        new_eucs_dict[eucs_id] = eucs_text
        
        # Parse mappings
        bsi_ids = parse_control_ids(row.get('C5.2020 GERMANY ', ''))
        secnum_ids = parse_control_ids(row.get('SecNumCloud FRANCE ', ''))
        iso27002_ids = parse_control_ids(row.get('ISO 27002 ', ''))
        iso27017_ids = parse_control_ids(row.get('ISO 27017 ', ''))
        
        if bsi_ids:
            new_eucs_to_bsi[eucs_id] = bsi_ids
        if secnum_ids:
            new_eucs_to_secnum[eucs_id] = secnum_ids
        if iso27002_ids:
            new_eucs_to_iso27002[eucs_id] = iso27002_ids
        if iso27017_ids:
            new_eucs_to_iso27017[eucs_id] = iso27017_ids

print(f"✓ Loaded NewEucsRequirements.csv: {len(new_eucs_dict)} controls")
print(f"  - BSI-C5 mappings: {len(new_eucs_to_bsi)}")
print(f"  - SecNumCloud mappings: {len(new_eucs_to_secnum)}")

# Load Medina Metrics
medina_df = pd.read_csv(FILES['medina_metrics'], encoding='utf-8')
medina_dict = {}
for _, row in medina_df.iterrows():
    metric_id = str(row['ID'])
    metric_desc = row['description']
    medina_dict[metric_id] = metric_desc
print(f"✓ Loaded medinaMetrics.csv: {len(medina_df)} metrics")

# Load Fabasoft Metrics
fabasoft_df = pd.read_csv(FILES['fabasoft_metrics'], encoding='utf-8')
fabasoft_dict = {}
for _, row in fabasoft_df.iterrows():
    metric_id = row['ID']
    if pd.notna(metric_id):
        metric_desc = row['Description']
        fabasoft_dict[metric_id] = metric_desc
print(f"✓ Loaded fabasoftMetrics.csv: {len(fabasoft_dict)} metrics")

# ============================================================================
# GENERATE TRAINING DATASET
# ============================================================================

print("\n" + "="*70)
print("GENERATING TRAINING DATASET")
print("="*70)

training_pairs = []

# 1. SpanishENS ↔ Cisco
print("\n1. Processing SpanishENS ↔ Cisco...")
for _, row in cisco_df.iterrows():
    cisco_id = standardize_id(row['Control Reference'])
    cisco_text = row['Control Wording']
    
    # Get Spanish ENS mappings from three columns (BASIC, Medium, High)
    spanish_ids = []
    for col in ['Spanish ENS BASIC Control', 'Spanish ENS Medium Control', 'Spanish ENS High Control']:
        spanish_ids.extend(parse_control_ids(row.get(col, '')))
    
    if spanish_ids:
        pairs = create_pairs(cisco_id, cisco_text, spanish_ids, spanish_ens_dict)
        training_pairs.extend(pairs)

print(f"   Added {len([p for p in training_pairs if p['source_id'].startswith('CCF')])} pairs")

# 2. NewEucsRequirements ↔ Cisco
print("2. Processing NewEucsRequirements ↔ Cisco...")
initial_count = len(training_pairs)
for _, row in cisco_df.iterrows():
    cisco_id = standardize_id(row['Control Reference'])
    cisco_text = row['Control Wording']
    
    # Get EUCS mappings from three columns (Basic, Substantial, High)
    eucs_ids = []
    for col in ['EUCS Basic Control', 'EUCS Substantial Control', 'EUCS High Control']:
        eucs_ids.extend(parse_control_ids(row.get(col, '')))
    
    if eucs_ids:
        pairs = create_pairs(cisco_id, cisco_text, eucs_ids, new_eucs_dict)
        training_pairs.extend(pairs)

print(f"   Added {len(training_pairs) - initial_count} pairs")

# 3. Secnumcloud ↔ Cisco
print("3. Processing Secnumcloud ↔ Cisco...")
initial_count = len(training_pairs)
for _, row in cisco_df.iterrows():
    cisco_id = standardize_id(row['Control Reference'])
    cisco_text = row['Control Wording']
    
    secnum_ids = parse_control_ids(row.get('SecNumCloud Control', ''))
    
    if secnum_ids:
        pairs = create_pairs(cisco_id, cisco_text, secnum_ids, secnumcloud_dict)
        training_pairs.extend(pairs)

print(f"   Added {len(training_pairs) - initial_count} pairs")

# 4. BSI-C5 ↔ Cisco
print("4. Processing BSI-C5 ↔ Cisco...")
initial_count = len(training_pairs)
for _, row in cisco_df.iterrows():
    cisco_id = standardize_id(row['Control Reference'])
    cisco_text = row['Control Wording']
    
    bsi_ids = parse_control_ids(row.get('BSI C5', ''))
    
    if bsi_ids:
        pairs = create_pairs(cisco_id, cisco_text, bsi_ids, bsi_c5_dict)
        training_pairs.extend(pairs)

print(f"   Added {len(training_pairs) - initial_count} pairs")

# 5. BSI-C5 ↔ fabasoftMetrics
print("5. Processing BSI-C5 ↔ fabasoftMetrics...")
initial_count = len(training_pairs)
for _, row in fabasoft_df.iterrows():
    metric_id = row['ID']
    if pd.isna(metric_id):
        continue
        
    metric_desc = row['Description']
    
    # Parse BSI C5 control IDs from "Possible Control ID Scheme" column
    control_scheme = row.get('Possible Control ID\nScheme', '')
    bsi_ids = parse_control_ids(control_scheme)
    
    if bsi_ids:
        pairs = create_pairs(metric_id, metric_desc, bsi_ids, bsi_c5_dict)
        training_pairs.extend(pairs)

print(f"   Added {len(training_pairs) - initial_count} pairs")

# 6. BSI-C5 ↔ NewEucsRequirements (only for IDs NOT in OldEucsRequirements)
print("6. Processing BSI-C5 ↔ NewEucsRequirements (excluding old IDs)...")
initial_count = len(training_pairs)
for eucs_id, bsi_ids in new_eucs_to_bsi.items():
    # Only include if this EUCS ID is NOT in old requirements
    if eucs_id not in old_eucs_ids:
        eucs_text = new_eucs_dict.get(eucs_id, '')
        if eucs_text:
            pairs = create_pairs(eucs_id, eucs_text, bsi_ids, bsi_c5_dict)
            training_pairs.extend(pairs)

print(f"   Added {len(training_pairs) - initial_count} pairs")

# 7. fabasoftMetrics ↔ NewEucsRequirements
print("7. Processing fabasoftMetrics ↔ NewEucsRequirements...")
initial_count = len(training_pairs)
for _, row in fabasoft_df.iterrows():
    metric_id = row['ID']
    if pd.isna(metric_id):
        continue
        
    metric_desc = row['Description']
    
    # Parse EUCS control IDs from "Possible Control ID Scheme" column
    control_scheme = row.get('Possible Control ID\nScheme', '')
    
    # Check if control_scheme is valid (not NaN or empty)
    if pd.isna(control_scheme) or control_scheme == '':
        continue
    
    # Look for EUCS pattern (e.g., "BSI C5 OPS-22" or just "OPS-22")
    eucs_ids = []
    for line in str(control_scheme).split('\n'):
        # Extract IDs that match EUCS pattern
        ids = parse_control_ids(line)
        for id_str in ids:
            if id_str in new_eucs_dict:
                eucs_ids.append(id_str)
    
    if eucs_ids:
        pairs = create_pairs(metric_id, metric_desc, eucs_ids, new_eucs_dict)
        training_pairs.extend(pairs)

print(f"   Added {len(training_pairs) - initial_count} pairs")

# ============================================================================
# GENERATE TEST DATASET
# ============================================================================

print("\n" + "="*70)
print("GENERATING TEST DATASET")
print("="*70)

test_pairs = []

# 1. BSI-C5 ↔ OldEucsRequirements (all of them)
print("\n1. Processing BSI-C5 ↔ OldEucsRequirements...")
# We need to find BSI-C5 to Old EUCS mappings
# These come from the OldEucsRequirements associations in NewEucsRequirements
for eucs_id in old_eucs_ids:
    if eucs_id in new_eucs_to_bsi:
        eucs_text = old_eucs_dict.get(eucs_id, '')
        bsi_ids = new_eucs_to_bsi[eucs_id]
        
        if eucs_text:
            pairs = create_pairs(eucs_id, eucs_text, bsi_ids, bsi_c5_dict)
            test_pairs.extend(pairs)

print(f"   Added {len(test_pairs)} pairs")

# 2. medinaMetrics ↔ OldEucsRequirements (all of them)
print("2. Processing medinaMetrics ↔ OldEucsRequirements...")
initial_count = len(test_pairs)
for _, row in medina_df.iterrows():
    metric_id = str(row['ID'])
    metric_desc = row['description']
    control_id = standardize_id(row['controlId'])
    
    if control_id in old_eucs_dict:
        control_text = old_eucs_dict[control_id]
        test_pairs.append({
            'source_id': metric_id,
            'source_text': metric_desc,
            'target_id': control_id,
            'target_text': control_text,
            'label': 1
        })

print(f"   Added {len(test_pairs) - initial_count} pairs")

# ============================================================================
# SAVE DATASETS
# ============================================================================

print("\n" + "="*70)
print("SAVING DATASETS")
print("="*70)

# Convert to DataFrames
train_df = pd.DataFrame(training_pairs)
test_df = pd.DataFrame(test_pairs)

# Remove duplicates
train_df = train_df.drop_duplicates(subset=['source_id', 'target_id'])
test_df = test_df.drop_duplicates(subset=['source_id', 'target_id'])

# Save to CSV
train_df.to_csv(OUTPUT['train'], index=False, encoding='utf-8')
test_df.to_csv(OUTPUT['test'], index=False, encoding='utf-8')

print(f"\n✓ Training dataset saved: {OUTPUT['train']}")
print(f"  Total pairs: {len(train_df)}")
print(f"  Unique source IDs: {train_df['source_id'].nunique()}")
print(f"  Unique target IDs: {train_df['target_id'].nunique()}")

print(f"\n✓ Test dataset saved: {OUTPUT['test']}")
print(f"  Total pairs: {len(test_df)}")
print(f"  Unique source IDs: {test_df['source_id'].nunique()}")
print(f"  Unique target IDs: {test_df['target_id'].nunique()}")

# Display samples
print("\n" + "="*70)
print("TRAINING DATASET SAMPLES")
print("="*70)
print(train_df.head(10))

print("\n" + "="*70)
print("TEST DATASET SAMPLES")
print("="*70)
print(test_df.head(10))

print("\n✅ Dataset generation complete!")

Loading data files...
✓ Loaded Cisco.csv: 713 rows
✓ Loaded SpanishENS.csv: 204 controls
✓ Loaded Secnumcloud.csv: 261 controls
✓ Loaded BSI-C5.json: 222 controls
✓ Loaded OldEucsRequirements.csv: 70 controls
✓ Loaded NewEucsRequirements.csv: 522 controls
  - BSI-C5 mappings: 492
  - SecNumCloud mappings: 366
✓ Loaded medinaMetrics.csv: 183 metrics
✓ Loaded fabasoftMetrics.csv: 57 metrics

GENERATING TRAINING DATASET

1. Processing SpanishENS ↔ Cisco...
   Added 9 pairs
2. Processing NewEucsRequirements ↔ Cisco...
   Added 1277 pairs
3. Processing Secnumcloud ↔ Cisco...
   Added 568 pairs
4. Processing BSI-C5 ↔ Cisco...
   Added 554 pairs
5. Processing BSI-C5 ↔ fabasoftMetrics...
   Added 27 pairs
6. Processing BSI-C5 ↔ NewEucsRequirements (excluding old IDs)...
   Added 564 pairs
7. Processing fabasoftMetrics ↔ NewEucsRequirements...
   Added 0 pairs

GENERATING TEST DATASET

1. Processing BSI-C5 ↔ OldEucsRequirements...
   Added 0 pairs
2. Processing medinaMetrics ↔ OldEucsRequiremen